# 📘 DevOps 업무전환 - Story 포인트 측정
- **목적**: DevOps 전환을 위해 생성된 스토리에서 스토리포인트 산정
- **입력 데이터**: generate_story_of_devops.ipynb의 결과물
- **주요 기능**
  - 스토리 포인트 참고 자료 로드
  - 스토리별 스토리 포인트 산정 
- **작성자**: 김동영
- **작성일**: 2025-08-05

# 0. 환경 설정

In [1]:
import os, sys
sys.path.append(os.path.abspath("../src"))

import dotenv
import json

from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

from story_point.services import StoryPointEstimationAgent
# from epic.prompts import TASK_TO_EPIC_CONVERTER_PROMPT

In [2]:
# .env 환경번수 load
dotenv.load_dotenv('../.env')

True

# 1. Story Point reference load and Agent 인스턴스 생성

In [3]:
# 스토리 생성 llm 모델 생성
story_point_agent = StoryPointEstimationAgent(
    openai_api_key = os.getenv("OPENAI_API_KEY"),csv_file_path='../data/reference_stories_250805.csv'
)

/Users/visuworks/Documents/07_n8n/project_manage_agent/src/story_point/services.py:21: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  self.llm = ChatOpenAI(
INFO:story_point.services:참고 데이터 로드 완료: 5개 스토리


In [4]:
STORY_POINT_ESTIMATION_PROMPT = ChatPromptTemplate.from_messages(
    [SystemMessagePromptTemplate.from_template(
        """
        [Role] 너는 숙련된 스크럼 마스터다.
        [Goal] 2주 단위 스프린트를 기반으로 한 애자일 방법론에서, 주어진 에픽정보와 Story를 기반으로 각 Story의 업무 리소스 산정 단위인 StoryPoint를 도출하고 업무 계획을 수립하는 데 도움을 주는 것.
        [Instruction] 주어진 Epic 정보와 Story 내용을 기반으로, 각 Story의 StoryPoint를 추정하고, 그 이유를 상세하게 설명해.
        [Constraints]
          - 출력은 반드시 [Output Format]을 따라야 한다.
          - StoryPoint는 반드시 1, 2, 3, 5, 8 중 하나의 값을 가진다.
          - 불필요한 설명, 주석, 마크다운 없이 JSON만 출력한다.
          - Story가 1개이더라도 반드시 [Output Format]과 동일한 JSON 배열 형식으로 출력한다.
          - 스토리 포인트 추정 기준
            - **1 포인트**: 매우 간단한 작업 (1-2시간)
              - 단순 설정 변경, 텍스트 수정, 기존 기능 복사
            - **2 포인트**: 간단한 작업 (반나절)
              - 간단한 CRUD 작업, 기존 컴포넌트 재사용
            - **3 포인트**: 보통 작업 (1일, 6-8시간)
              - 새로운 화면 개발, 중간 복잡도의 비즈니스 로직
            - **5 포인트**: 복잡한 작업 (2-3일, 12-24시간)
              - 복잡한 비즈니스 로직, 외부 시스템 연동, 여러 컴포넌트 간 협업
            - **8 포인트**: 매우 복잡한 작업 (1주일, 32-40시간)
              - 아키텍처 변경, 대규모 리팩토링, 신규 기술 도입, 높은 불확실성
        [Process]
          1 → 2 → 3 → 4의 순서로 Story의 복잡도를 분석하고 추정 방법을 선택한 뒤, 최종 StoryPoint를 결정한다.
          1. 참고 스토리 분석
            - 같은 영역의 참고 스토리가 있는 경우: **직접 비교**
            - 같은 영역의 참고 스토리가 없는 경우: **다른 영역 스토리 활용**
            - 참고할 스토리가 없는 경우: **해당 스토리를 분석하여 임의로 산정**

          2. 복잡도 분석
            다음 요소들을 종합적으로 평가
            2.1 기술적 복잡도
              - 구현 난이도 (새로운 기술/라이브러리 사용 여부)
              - 코드 변경 범위 (신규 개발 vs 기존 수정)
              - 테스트 복잡도 (단위/통합/E2E 테스트 범위)
              - 성능 고려사항 (최적화 필요 여부)
            2.2 비즈니스 복잡도
              - 요구사항 명확성 및 완성도
              - 비즈니스 로직 복잡도
              - 사용자 인터페이스 복잡도
              - 데이터 처리 복잡도
              - 인수 조건(acceptance_criteria) 복잡도 및 개수
            2.3 위험 요소
              - 불확실성 (요구사항 변경 가능성)
              - 의존성 (다른 팀/시스템과의 의존도)
              - 학습 곡선 (새로운 기술/도메인 학습 필요)
              - 블로커 가능성 (외부 요인으로 인한 지연 위험)

          3. 추정 방법 선택
            Case1 직접 비교 (같은 영역 스토리가 있는 경우)
              1. 같은 영역의 참고 스토리들과 복잡도 비교
              2. 가장 유사한 스토리 식별
              3. 상대적 차이를 고려하여 포인트 조정
            Case2 다른 영역 스토리 활용 (같은 영역 스토리가 없는 경우)
              1. 모든 영역의 스토리 중 유사한 복잡도 패턴 찾기
              2. 영역 차이를 고려한 복잡도 보정
              3. 기술적/비즈니스적 유사성이 높은 스토리 우선 참조

          4. 최종 포인트 결정
            - 반드시 피보나치 수열 값(1, 2, 3, 5, 8) 중 선택
            - 두 포인트 사이에서 애매한 경우 더 높은 포인트 선택 (안전 마진)

        [Output Format]
         [
          {{
            "story_title": "스토리 제목",
            "estimated_point": 3,
            "domain": "frontend|backend|devops|data",
            "estimation_method": "same_area|cross_area",
            "reasoning": "상세한 추정 근거 (왜 이 포인트인지 논리적 설명)",
            "complexity_factors": [
              "기술적 복잡도 요소",
              "비즈니스 복잡도 요소",
              "위험 요소"
            ],
            "similar_stories": [
              "유사한 참고 스토리 제목들 (영역 구분 없이)"
            ],
            "confidence_level": "high|medium|low",
            "assumptions": [
              "추정 시 가정한 사항들"
            ],
            "risks": [
              "예상되는 위험 요소들"
            ]
          }}
        ]
        """
    ),
    HumanMessagePromptTemplate.from_template(
        """
        [Input] {user_input}
        [Context]
          epic info : {epic_info}
          story info : {story_info}
          reference_stories : {reference_stories}
        """
    )
    ]
)


In [5]:
story_point_agent.csv_file_path

'../data/reference_stories_250805.csv'

In [6]:
ci_cd_story_list = [
  {
    "title": "CI/CD 자동화 도구 선택",
    "description": "GitHub Actions 또는 Jenkins 중 하나를 선택하여 CI/CD 자동화 도구를 결정한다.",
    "acceptance_criteria": [
      "CI/CD 도구 선택에 대한 문서화가 완료된다.",
      "선택된 도구의 장단점 분석이 포함된다."
    ],
    "domain": "devops",
    "story_type": "feature",
    "tags": ["CI/CD", "GitHub Actions", "Jenkins"]
  },
  {
    "title": "CI/CD 파이프라인 기본 구성",
    "description": "선택된 CI/CD 도구를 사용하여 기본 파이프라인을 구성한다.(Bnviit IOL 기준)",
    "acceptance_criteria": [
      "기본 파이프라인이 성공적으로 실행된다.",
      "파이프라인 구성에 대한 문서가 작성된다."
    ],
    "domain": "devops",
    "story_type": "feature",
    "tags": ["CI/CD", "파이프라인", "자동화"]
  },
  {
  "title": "PR 시 자동 코드 검증 및 품질 점검 설정",
  "description": "GitHub Actions 등 CI 도구를 통해 PR 생성 시 Lint, Formatting, Unit Test, Build, Code Quality Check가 자동으로 실행되도록 설정한다.(Bnviit IOL 기준)",
  "acceptance_criteria": [
    "PR 생성 시 Lint가 자동으로 실행된다.",
    "PR 생성 시 코드 Formatting이 자동으로 검증된다.",
    "PR 생성 시 Unit Test가 자동으로 실행된다.",
    "PR 생성 시 Build가 자동으로 실행된다.",
    "PR 생성 시 Code Quality Check가 자동으로 실행된다."
  ],
  "domain": "devops",
  "story_type": "feature",
  "tags": ["CI/CD", "Lint", "Unit Test", "Build", "Code Quality", "GitHub Actions"]
},
  {
    "title": "Dockerfile 오류 및 종속성 검증 설정",
    "description": "Dockerfile 실행 오류 및 종속성 검증을 CI/CD 파이프라인에 포함시킨다.(Bnviit IOL 기준)",
    "acceptance_criteria": [
      "Dockerfile 실행 오류가 감지될 경우 알림이 발생한다.",
      "종속성 검증이 성공적으로 수행된다."
    ],
    "domain": "devops",
    "story_type": "feature",
    "tags": ["Docker", "CI/CD", "검증"]
  },
  {
    "title": "CI/CD 결과 알림 설정",
    "description": "CI/CD 결과를 Slack, 이메일, 웹훅으로 알림받도록 설정한다.(Bnviit IOL 기준)",
    "acceptance_criteria": [
      "CI/CD 결과가 Slack으로 성공적으로 전송된다.",
      "CI/CD 결과가 이메일로 성공적으로 전송된다."
    ],
    "domain": "devops",
    "story_type": "feature",
    "tags": ["알림", "Slack", "이메일"]
  },
  {
    "title": "비앤빛 비젼(Java) 프로젝트에 CI/CD 적용",
    "description": "비앤빛 비젼(Java 기반)의 백엔드 프로젝트에 CI/CD 자동화 구성을 적용한다. Java 프로젝트 특성에 맞게 Maven/Gradle 기반 빌드 및 테스트 설정을 포함한다.",
    "acceptance_criteria": [
      "Java 기반 빌드 및 테스트 파이프라인이 구성된다.",
      "PR 생성 시 자동으로 Lint, Test, Build가 실행된다.",
      "Slack 또는 이메일을 통한 알림이 설정된다."
    ],
    "domain": "devops",
    "story_type": "feature",
    "tags": ["CI/CD", "Java", "비앤빛비젼", "GitHub Actions", "Maven", "Gradle"]
  },
  {
    "title": "브리즘 마이오피아 프로젝트에 CI/CD 적용",
    "description": "브리즘 마이오피아 프로젝트에 맞는 CI/CD 파이프라인을 구성하고, Python 기반 프로젝트 특성에 맞는 Lint, Test, Build 설정을 포함한다.",
    "acceptance_criteria": [
      "Python 기반 빌드 및 테스트 파이프라인이 구성된다.",
      "PR 생성 시 자동으로 Lint, Test, Build가 실행된다.",
      "CI 결과가 Slack 또는 이메일로 전송된다."
    ],
    "domain": "devops",
    "story_type": "feature",
    "tags": ["CI/CD", "Python", "브리즘", "GitHub Actions"]
  },
  {
    "title": "비앤빛 OCR 프로젝트에 CI/CD 적용",
    "description": "비앤빛 OCR 서비스에 CI/CD 파이프라인을 구축하고, 모델 추론 로직의 검증과 이미지 처리 테스트를 자동화한다.",
    "acceptance_criteria": [
      "OCR 모델 추론에 대한 테스트가 CI에 포함된다.",
      "PR 생성 시 Lint, Unit Test, Build가 자동 실행된다.",
      "CI/CD 결과에 대한 알림이 설정된다."
    ],
    "domain": "devops",
    "story_type": "feature",
    "tags": ["CI/CD", "OCR", "이미지처리", "비앤빛OCR", "GitHub Actions"]
  },
  {
    "title": "비앤빛 App 프로젝트에 CI/CD 적용",
    "description": "비앤빛 App (모바일 또는 웹 프론트) 프로젝트에 CI/CD 자동화 설정을 적용하고, 프론트엔드 빌드 및 테스트 수행을 포함한다.",
    "acceptance_criteria": [
      "프론트엔드 빌드 및 테스트가 자동으로 실행된다.",
      "PR 생성 시 Lint, Test, Build가 포함된다.",
      "Slack 알림 또는 GitHub status check가 설정된다."
    ],
    "domain": "devops",
    "story_type": "feature",
    "tags": ["CI/CD", "프론트엔드", "비앤빛App", "React", "GitHub Actions"]
  }
]

In [9]:
ci_cd_epic_info ={
    "title": "CI/CD 자동화 구성",
    "description": "GitHub Actions 또는 Jenkins를 활용하여 End-to-End CI/CD 자동화 프로세스를 구성한다.",
    "business_value": "개발 및 배포 프로세스의 효율성을 높이고, 코드 품질을 유지하여 빠른 피드백을 제공한다.",
    "priority": "High",
    "acceptance_criteria": [
      "PR 생성 시 Lint, Unit Test, Build, Code Quality Check가 자동으로 실행된다.",
      "Dockerfile 실행 오류 및 종속성 검증이 포함된다.",
      "CI/CD 결과가 Slack/이메일/웹훅으로 알림된다."
    ],
    "included_tasks": [
      "GitHub Actions 또는 Jenkins를 활용한 End-to-End 자동화 구성",
      "PR 생성 시 Lint, Unit Test, Build, Code Quality Check 자동 실행",
      "Dockerfile 실행 오류 및 종속성 검증 포함",
      "CI/CD 결과를 Slack/이메일/웹훅 알림으로 연동"
    ]
  },

In [10]:
story_point_agent.get_reference_stories_by_domain('DevOps')

[{'story_title': 'PR 생성 시 자동 테스트 실행',
  'description': 'PR 생성 시 자동으로 Lint, Unit Test 작업 실행하도록 GitHub Action 설정',
  'domain': 'DevOps',
  'estimated_point': 5,
  'reasoning': 'PR 발생 시 GitHub Actions를 트리거하여 Lint 및 Unit Test를 자동 수행하도록 .github/workflows/ci.yml 워크플로 파일 구성. PR 품질 체크를 자동화하고 merge 조건으로 연결.',
  'complexity_factors': 'nan',
  'confidence_level': 'low'},
 {'story_title': '코드 빌드 자동화 및 Docker 이미지 테스트',
  'description': 'GitHub Actions 내 Dockerfile 빌드 테스트 자동화 설정. 코드 변경 시 도커 이미지가 정상 빌드되고 필요한 경우 레지스트리(Public/Private)에 업로드 전 확인 가능',
  'domain': 'DevOps',
  'estimated_point': 5,
  'reasoning': 'Docker 환경 및 종속성 처리 여부에 따라 에러가 발생할 수 있어 디버깅 시간이 필요. 빌드 캐시 전략이나 멀티 스테이지 고려 시 작업량이 늘어남.',
  'complexity_factors': 'nan',
  'confidence_level': 'low'},
 {'story_title': 'DockerHub로 CI 결과물 자동 푸시',
  'description': '빌드 완료 후 DockerHub로 latest, version 태그를 붙여 이미지 푸시. GitHub Secret에 인증 정보 설정 포함.',
  'domain': 'DevOps',
  'estimated_point': 3,
  'reasoning': '빌드 후 업로드만 하면 되므로 로직은 단순하지만, GitHub Secrets 관리와 

In [11]:
story_point_by_story = {}

for story in ci_cd_story_list[:]:
    story_point_by_story[story['title']] = {}

    estimated_story_point_str = story_point_agent._generate_estimations_with_llm(
        prompt = STORY_POINT_ESTIMATION_PROMPT,
        user_input = '',
        story_info= story,
        epic_info=ci_cd_epic_info,
        reference_stories=story_point_agent.get_reference_stories_by_domain('DevOps')
    )
    parsed = json.loads(estimated_story_point_str)
    story_point_by_story[story['title']] = parsed


INFO:story_point.services:스토리 포인트 추정 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:story_point.services:스토리 포인트 추정 완료
INFO:story_point.services:스토리 포인트 추정 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:story_point.services:스토리 포인트 추정 완료
INFO:story_point.services:스토리 포인트 추정 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:story_point.services:스토리 포인트 추정 완료
INFO:story_point.services:스토리 포인트 추정 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:story_point.services:스토리 포인트 추정 완료
INFO:story_point.services:스토리 포인트 추정 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:story_point.services:스토리 포인트 추정 완료
INFO:story_point.services:스토리 포인트 추정 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:story_poi

In [12]:
story_point_by_story

{'CI/CD 자동화 도구 선택': [{'story_title': 'CI/CD 자동화 도구 선택',
   'estimated_point': 3,
   'domain': 'devops',
   'estimation_method': 'same_area',
   'reasoning': 'CI/CD 도구를 선택하는 작업은 기술적 분석과 문서화가 필요하지만, 기존의 도구들(GitHub Actions, Jenkins)에 대한 이해가 필요하여 복잡도가 보통 수준입니다. 선택된 도구의 장단점을 분석하는 과정이 포함되어 있어, 단순한 결정 이상의 작업이 요구됩니다.',
   'complexity_factors': ['기술적 복잡도: 도구 선택에 대한 기술적 분석 필요',
    '비즈니스 복잡도: 선택된 도구의 장단점 문서화',
    '위험 요소: 도구 선택에 따른 향후 변경 가능성'],
   'similar_stories': ['PR 생성 시 자동 테스트 실행',
    '코드 빌드 자동화 및 Docker 이미지 테스트',
    'DockerHub로 CI 결과물 자동 푸시'],
   'confidence_level': 'medium',
   'assumptions': ['선택할 도구에 대한 기본적인 이해가 있다고 가정', '문서화 작업이 명확하게 정의되어 있다고 가정'],
   'risks': ['도구 선택 후 요구사항 변경 가능성', '선택된 도구의 기술적 문제 발생 가능성']}],
 'CI/CD 파이프라인 기본 구성': [{'story_title': 'CI/CD 파이프라인 기본 구성',
   'estimated_point': 5,
   'domain': 'devops',
   'estimation_method': 'same_area',
   'reasoning': '기본 파이프라인 구성은 여러 작업을 포함하며, CI/CD 도구의 설정 및 문서화가 필요하다. PR 생성 시 자동 테스트 실행과 코드 빌드 자동화와 유사한 복잡도를 가지며, 성공적인 실행을 보장하기 위해 다